# Generated by alkh at 2023-03-08 19:02:14

In [1]:
import sys
sys.path.insert(0, r'/mnt/D/tech/repos/alkh/tests')
sys.path.insert(1, r'/snap/pycharm-community/315/plugins/python-ce/helpers/pydev')
sys.path.insert(2, r'/snap/pycharm-community/315/plugins/python-ce/helpers/third_party/thriftpy')
sys.path.insert(3, r'/snap/pycharm-community/315/plugins/python-ce/helpers/pydev')
sys.path.insert(4, r'/mnt/D/tech/repos/alkh')
sys.path.insert(5, r'/home/erez/.cache/JetBrains/PyCharmCE2022.3/cythonExtensions')
sys.path.insert(6, r'/mnt/D/tech/repos/alkh/tests')
sys.path.insert(7, r'/home/erez/anaconda3/lib/python39.zip')
sys.path.insert(8, r'/home/erez/anaconda3/lib/python3.9')
sys.path.insert(9, r'/home/erez/anaconda3/lib/python3.9/lib-dynload')
sys.path.insert(10, r'/home/erez/anaconda3/lib/python3.9/site-packages')
sys.path.insert(11, r'/home/erez/anaconda3/lib/python3.9/site-packages/IPython/extensions')

In [2]:
import pandas as pd
import alkh

In [3]:
from alkh.logic_core import CallGraphManager
from alkh.utils import code_range_utils
import libcst as cst
from typing import *
import networkx as nx
import itertools
import numpy as np

In [4]:
stack_df = pd.read_pickle('13ae1b84e1014b7cb46d666553542658.pkl')
stack_df[["file_path", "function", "lineno", "locals_names"]]

,file_path,function,lineno,locals_names
0,/mnt/D/tech/repos/alkh/alkh/logic_core.py,get_lines_numbers_affecting_line_number,32,"[self, line_number, target_id_to_line_numbers_..."
1,/mnt/D/tech/repos/alkh/tests/logic_core_test.py,<module>,7,"[file_path, line_number]"


In [5]:
alkh.print_context(stack_df.loc[0, 'context'])

/mnt/D/tech/repos/alkh/alkh/logic_core.py
29:             final_lines_numbers_list = [line_number]
30: 
31:         return final_lines_numbers_list
32: 
33:     def _get_lines_numbers_affecting_line_number_with_targets(self, line_targets_list, target_id_to_line_numbers_df):


In [6]:
self = stack_df.loc[0, 'locals']['self']
line_number = stack_df.loc[0, 'locals']['line_number']
target_id_to_line_numbers_df = stack_df.loc[0, 'locals']['target_id_to_line_numbers_df']
lines_contains_series = stack_df.loc[0, 'locals']['lines_contains_series']
line_targets_list = stack_df.loc[0, 'locals']['line_targets_list']
final_lines_numbers_list = stack_df.loc[0, 'locals']['final_lines_numbers_list']

In [7]:
final_lines_numbers_list

[10, 11, 12, 13]

In [8]:
self._calls_df

,target,node_range,scope_index
0,"(self, a)","CodeRange(start=CodePosition(line=3, column=8)...",1
1,"(a_func,)","CodeRange(start=CodePosition(line=7, column=8)...",5
2,"(self, a)","CodeRange(start=CodePosition(line=12, column=1...",4
3,"(a_func,)","CodeRange(start=CodePosition(line=16, column=8...",5


In [10]:
call_series = self._calls_df.iloc[0]
call_series

target                                                 (self, a)
node_range     CodeRange(start=CodePosition(line=3, column=8)...
scope_index                                                    1
Name: 0, dtype: object

In [11]:
node_range = call_series["node_range"]
node_range

CodeRange(start=CodePosition(line=3, column=8), end=CodePosition(line=3, column=16))

In [20]:
any([node_range.start.line <= line <= node_range.end.line for line in final_lines_numbers_list])

False

In [22]:
def is_call_in_lines(call_range, final_lines_numbers_list):
    result = any([node_range.start.line <= line <= call_range.end.line for line in final_lines_numbers_list])
    return result

In [26]:
is_in_lines_series = self._calls_df["node_range"].apply(is_call_in_lines, args=(final_lines_numbers_list,))
is_in_lines_series

0    False
1    False
2     True
3     True
Name: node_range, dtype: bool

In [29]:
a = self._calls_df[is_in_lines_series]
a

,target,node_range,scope_index
2,"(self, a)","CodeRange(start=CodePosition(line=12, column=1...",4
3,"(a_func,)","CodeRange(start=CodePosition(line=16, column=8...",5


In [31]:
a[["scope_index"]].merge(self._scopes_df)

,scope_index,scope,start_line_number,end_line_number,header_end_line_number,length,name,node_range
0,4,<libcst.metadata.scope_provider.FunctionScope ...,15,17,15,3,a,"CodeRange(start=CodePosition(line=15, column=4..."
1,5,<libcst.metadata.scope_provider.FunctionScope ...,20,21,20,2,a_func,"CodeRange(start=CodePosition(line=20, column=0..."


In [28]:
self._scopes_df

,scope,start_line_number,end_line_number,header_end_line_number,length,name,node_range,scope_index
0,<libcst.metadata.scope_provider.FunctionScope ...,2,4,2,3,__init__,"CodeRange(start=CodePosition(line=2, column=4)...",0
1,<libcst.metadata.scope_provider.FunctionScope ...,6,7,6,2,a,"CodeRange(start=CodePosition(line=6, column=4)...",1
2,<libcst.metadata.scope_provider.ClassScope obj...,10,17,10,8,B,"CodeRange(start=CodePosition(line=10, column=0...",2
3,<libcst.metadata.scope_provider.ClassScope obj...,1,7,1,7,A,"CodeRange(start=CodePosition(line=1, column=0)...",3
4,<libcst.metadata.scope_provider.FunctionScope ...,15,17,15,3,a,"CodeRange(start=CodePosition(line=15, column=4...",4
5,<libcst.metadata.scope_provider.FunctionScope ...,20,21,20,2,a_func,"CodeRange(start=CodePosition(line=20, column=0...",5
6,<libcst.metadata.scope_provider.GlobalScope ob...,1,21,1,21,global,"CodeRange(start=CodePosition(line=1, column=0)...",6
7,<libcst.metadata.scope_provider.FunctionScope ...,11,13,11,3,__init__,"CodeRange(start=CodePosition(line=11, column=4...",7
